# 3. Flow Data

For this exercise we will use the **CTU-13 dataset, capture 51.**

In [1]:
%load_ext autoreload
%autoreload 2

In [7]:
import pandas as pd
import os
import sys
nb_dir = os.path.split(os.getcwd())[0]
if nb_dir not in sys.path:
    sys.path.append(nb_dir)
from global_helper import *
#from helper import *
from itertools import product
import seaborn as sb

Load data (Note: If the loading doesn't work, manually replace the header of capture 51 with the header from capture 43.)



In [3]:
#%%time

# Read file and preprocess columns (takes ~1.5min)
# Only loads the traffic associated with the provided IP address.
df = load_data("../data/capture20110818.pcap.netflow.labeled")


Below we have the summary of all the hosts

In [4]:
with pd.option_context("display.float_format", '{:.2f}'.format):
    display(df.describe())

,duration,tos,packets,bytes,flows
count,5180850.00,5180850.00,5180850.00,5180850.00,5180850.00
mean,0.50,0.05,12.26,9342.98,1.00
std,1.22,3.11,264.06,328644.16,0.00
min,0.00,0.00,1.00,60.00,1.00
25%,0.00,0.00,1.00,74.00,1.00
50%,0.00,0.00,1.00,133.00,1.00
75%,0.18,0.00,2.00,507.00,1.00
max,22.58,192.00,110189.00,166813983.00,1.00


Below we have the summary of the Botnet, followed by the summary of the legitimate hosts

In [5]:
botnet = df[df['label'] == 'Botnet']
#display(botnet[:20])
with pd.option_context("display.float_format", '{:.2f}'.format):
    display(botnet.describe())

,duration,tos,packets,bytes,flows
count,323441.00,323441.00,323441.00,323441.00,323441.00
mean,0.59,0.00,11.35,10365.96,1.00
std,1.46,0.00,80.64,66730.30,0.00
min,0.00,0.00,1.00,60.00,1.00
25%,0.00,0.00,1.00,1066.00,1.00
50%,0.00,0.00,1.00,1066.00,1.00
75%,0.00,0.00,1.00,1066.00,1.00
max,22.58,0.00,1002.00,788743.00,1.00


In [6]:
legitimate = df[df['label'] == 'LEGITIMATE']
#display(legitimate[:20])
with pd.option_context("display.float_format", '{:.2f}'.format):
    display(legitimate.describe())

,duration,tos,packets,bytes,flows
count,321917.00,321917.00,321917.00,321917.00,321917.00
mean,0.66,0.00,23.69,16593.96,1.00
std,1.34,0.00,154.19,177137.12,0.00
min,0.00,0.00,1.00,60.00,1.00
25%,0.00,0.00,1.00,66.00,1.00
50%,0.02,0.00,2.00,297.00,1.00
75%,0.39,0.00,7.00,2046.00,1.00
max,5.00,0.00,16937.00,24972468.00,1.00


Here we discretize the data, using #qcut.

In [8]:
df_actual = df[df['label'] != 'Background']
discretized = pd.qcut(df_actual['packets'], 5, duplicates='drop')
#display(discretized[:10])
#display(discretized.value_counts())

df_actual = df_actual[['label', 'protocol']]
df_actual = pd.concat([df_actual, discretized], axis=1)

display(df_actual[:10])


,label,protocol,packets
2011-08-18 10:19:13.347,LEGITIMATE,TCP,"(5.0, 16937.0]"
2011-08-18 10:19:13.392,LEGITIMATE,TCP,"(0.999, 2.0]"
2011-08-18 10:19:13.411,LEGITIMATE,TCP,"(5.0, 16937.0]"
2011-08-18 10:19:13.460,LEGITIMATE,TCP,"(5.0, 16937.0]"
2011-08-18 10:19:13.486,LEGITIMATE,TCP,"(0.999, 2.0]"
2011-08-18 10:19:13.689,LEGITIMATE,TCP,"(5.0, 16937.0]"
2011-08-18 10:19:13.848,LEGITIMATE,TCP,"(0.999, 2.0]"
2011-08-18 10:19:13.858,LEGITIMATE,TCP,"(2.0, 5.0]"
2011-08-18 10:19:14.067,LEGITIMATE,TCP,"(0.999, 2.0]"
2011-08-18 10:19:14.134,LEGITIMATE,TCP,"(0.999, 2.0]"


In [9]:
botnet_discretized = df_actual[df_actual['label'] == 'Botnet']
botnet_discretized[:10]

,label,protocol,packets
2011-08-18 10:20:40.906,Botnet,TCP,"(0.999, 2.0]"
2011-08-18 10:37:58.448,Botnet,UDP,"(0.999, 2.0]"
2011-08-18 10:37:58.448,Botnet,UDP,"(0.999, 2.0]"
2011-08-18 10:38:00.695,Botnet,UDP,"(0.999, 2.0]"
2011-08-18 10:38:00.705,Botnet,UDP,"(0.999, 2.0]"
2011-08-18 10:38:00.706,Botnet,TCP,"(2.0, 5.0]"
2011-08-18 10:38:00.714,Botnet,TCP,"(2.0, 5.0]"
2011-08-18 10:38:03.255,Botnet,UDP,"(0.999, 2.0]"
2011-08-18 10:38:03.266,Botnet,UDP,"(0.999, 2.0]"
2011-08-18 10:38:03.267,Botnet,UDP,"(0.999, 2.0]"


In [19]:
legitimate_discretized = df_actual[df_actual['label'] == 'LEGITIMATE']
legitimate_discretized[:10]

,label,protocol,packets
2011-08-18 10:19:13.347,LEGITIMATE,TCP,"(5.0, 16937.0]"
2011-08-18 10:19:13.392,LEGITIMATE,TCP,"(0.999, 2.0]"
2011-08-18 10:19:13.411,LEGITIMATE,TCP,"(5.0, 16937.0]"
2011-08-18 10:19:13.460,LEGITIMATE,TCP,"(5.0, 16937.0]"
2011-08-18 10:19:13.486,LEGITIMATE,TCP,"(0.999, 2.0]"
2011-08-18 10:19:13.689,LEGITIMATE,TCP,"(5.0, 16937.0]"
2011-08-18 10:19:13.848,LEGITIMATE,TCP,"(0.999, 2.0]"
2011-08-18 10:19:13.858,LEGITIMATE,TCP,"(2.0, 5.0]"
2011-08-18 10:19:14.067,LEGITIMATE,TCP,"(0.999, 2.0]"
2011-08-18 10:19:14.134,LEGITIMATE,TCP,"(0.999, 2.0]"


TO DO: Make heatmaps of the last two things and there you go!

In [13]:
legitimate_discretized2 = legitimate_discretized.groupby('protocol')
legitimate_discretized2[:10]

TypeError: unhashable type: 'slice'